## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-24-14-53-22 +0000,nyt,Ukrainians Are Grateful but Skeptical After Tr...,https://www.nytimes.com/2025/09/24/world/europ...
1,2025-09-24-14-46-56 +0000,bbc,At least one person killed in shooting at ICE ...,https://www.bbc.com/news/articles/ckge9d26z1ro...
2,2025-09-24-14-45-26 +0000,wapo,Live updates: One killed in shooting at ICE fa...,https://www.washingtonpost.com/nation/2025/09/...
3,2025-09-24-14-45-26 +0000,wapo,Assistant Homeland Security Secretary Tricia M...,https://www.washingtonpost.com
4,2025-09-24-14-44-34 +0000,nyt,U.N. General Assembly Updates: Zelensky Issues...,https://www.nytimes.com/live/2025/09/24/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2346/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,83
48,kimmel,25
47,jimmy,19
121,new,15
49,he,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
18,2025-09-24-14-05-20 +0000,nypost,Trump threatens to sue ABC over Jimmy Kimmel’s...,https://nypost.com/2025/09/24/us-news/trump-th...,162
173,2025-09-24-01-43-00 +0000,wsj,When Trump announced changes to the H-1B visa ...,https://www.wsj.com/politics/policy/air-india-...,156
148,2025-09-24-05-04-00 +0000,wsj,Kimmel Makes Emotional Return to ABC With Trum...,https://www.wsj.com/business/media/jimmy-kimme...,152
264,2025-09-23-19-35-00 +0000,wsj,President Trump attacked global immigration an...,https://www.wsj.com/politics/trump-says-migrat...,144
109,2025-09-24-09-30-00 +0000,wsj,Stephen Miran set out his case for lower inter...,https://www.wsj.com/economy/central-banking/st...,143


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
18,162,2025-09-24-14-05-20 +0000,nypost,Trump threatens to sue ABC over Jimmy Kimmel’s...,https://nypost.com/2025/09/24/us-news/trump-th...
173,73,2025-09-24-01-43-00 +0000,wsj,When Trump announced changes to the H-1B visa ...,https://www.wsj.com/politics/policy/air-india-...
140,59,2025-09-24-07-04-00 +0000,wsj,China said it will no longer seek new special ...,https://www.wsj.com/economy/trade/china-to-sto...
264,56,2025-09-23-19-35-00 +0000,wsj,President Trump attacked global immigration an...,https://www.wsj.com/politics/trump-says-migrat...
94,54,2025-09-24-10-00-00 +0000,nypost,Vivek Ramaswamy tells ‘Pod Force One’ he was o...,https://nypost.com/2025/09/24/us-news/vivek-ra...
43,50,2025-09-24-12-52-41 +0000,nypost,Dallas ICE facility shooting kills at least 2 ...,https://nypost.com/2025/09/24/us-news/sniper-c...
39,44,2025-09-24-13-01-18 +0000,cbc,"Zelenskyy addresses UN, as Russia scoffs at Tr...",https://www.cbc.ca/news/world/ukraine-war-zele...
123,42,2025-09-24-08-57-06 +0000,bbc,Jimmy Kimmel calls out 'anti-American' threats...,https://www.bbc.com/news/articles/c9dx6dlelw8o...
305,40,2025-09-23-17-29-24 +0000,nypost,Megyn Kelly slams ‘coward’ Van Jones for brand...,https://nypost.com/2025/09/23/media/megyn-kell...
291,39,2025-09-23-17-53-00 +0000,wsj,Fed Chair Jerome Powell said he judged the Fed...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
